In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(1, '../')

# Metadata Setup

For this notebook we will be building dataframes to hold any metadata information about CT scans that we need for training

In [3]:
import os
import feather
import random
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

from tqdm import tqdm
from pathlib import Path
from collections import namedtuple
Path.ls = lambda x: [o.name for o in x.iterdir()]

In [4]:
# Load environment variables to get local datasets path
from dotenv import load_dotenv
load_dotenv()
data_dir = os.environ.get('datasets_path')

Load the provided excel metadata and save paths to train and validation

In [5]:
dataset_path = Path(f'{data_dir}/COVID-19-20_v2')
trn_path = dataset_path/'Train'
val_path = dataset_path/'Validation'

In [6]:
trn_volumes = pd.read_excel(dataset_path/'COVID-19-20_TrainValidation.xlsx')

## File locations

First we organize the file path information in a dataframe for easy mapping between the IDs and their segmentation mask or CT volume

In [7]:
# train
trn_ct_fnames = list((trn_path/trn_volumes.FILENAME).astype(str) + '_ct.nii.gz')
trn_mask_fnames = list((trn_path/trn_volumes.FILENAME).astype(str) + '_seg.nii.gz')
trn_uid = list(trn_volumes.FILENAME.str[18:])

In [8]:
# valid
val_ct_list = pd.Series(val_path.ls())
val_ct_fnames = list((val_path/val_ct_list).astype(str))
val_uid = [val[18:23] if len(val) > 31 else val[18:21] for val in val_ct_list]

In [9]:
ct_fnames = trn_ct_fnames + val_ct_fnames
mask_fnames = trn_mask_fnames + ['']*len(val_ct_list)
uid = trn_uid + val_uid
is_valid = [False]*len(trn_ct_fnames) + [True]*len(val_ct_list)

In [10]:
df_meta = pd.DataFrame({'uid': uid,
                        'ct_fname': ct_fnames,
                        'mask_fname': mask_fnames,
                        'is_valid': is_valid})

In [11]:
df_meta.head()

,uid,ct_fname,mask_fname,is_valid
0,391,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,False
1,383_1,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,False
2,338,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,False
3,623,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,False
4,559,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,/mnt/16614710-4b50-44ce-a3f3-13105cdf14ad/data...,False


Assuming the `_1` or `_0` suffix on the id mean that it is either a duplicate or additional scan, we can drop the suffix and treat it as the same patient so we can keep it in the same train/validation set later on

In [12]:
df_meta['stripped_uid'] = df_meta.uid.str.replace('_1', '')
df_meta['stripped_uid'] = df_meta.stripped_uid.str.replace('_0', '')
len(df_meta.stripped_uid) > len(set(df_meta.stripped_uid))

False

Turns out the suffixes were not indicative of duplicates/additional scans for a patient, at least not for our training set

In [13]:
df_meta.drop('stripped_uid', axis=1, inplace=True)

## CT Header Info

Add the metadata from the header of each CT scan to df_meta for exploratory data analysis (EDA) purposes

In [14]:
# note that nibabel's get_fdata() function will cast the datatype of the ct array to float automatically
# so we do not have to handle any explicit data conversions
def append_header_meta(df, ct_fnames, mask_fnames):
    meta_dict = {}
    fnames = zip(ct_fnames, mask_fnames)
    for _, (ct_fname, mask_fname) in tqdm(enumerate(fnames), total=len(ct_fnames)):
        ct = nib.load(ct_fname)
        hdr = ct.header
        header_cols = set(list(hdr))
        for key in header_cols:
            if hdr[key].size > 1:
                for i, value in enumerate(hdr[key]):
                    meta_dict.setdefault(f'{key}_{i}',[]).append(value)
            else:
                meta_dict.setdefault(key,[]).append(hdr[key].item())
        img = ct.get_fdata()
        meta_dict.setdefault('img_min',[]).append(img.min())
        meta_dict.setdefault('img_max',[]).append(img.max())
        meta_dict.setdefault('img_mean',[]).append(img.mean())
        meta_dict.setdefault('img_std',[]).append(img.std())
        
        if len(mask_fname):
            mask = nib.load(mask_fname)
            img_mask = mask.get_fdata()
            meta_dict.setdefault('pct_lesion',[]).append(img_mask.sum() / img_mask.size)
        else:
            meta_dict.setdefault('pct_lesion',[]).append(0.)
        
    for key in meta_dict:
        df[key] = meta_dict[key]

In [15]:
append_header_meta(df_meta, df_meta.ct_fname, df_meta.mask_fname)

100%|██████████| 249/249 [01:38<00:00,  2.53it/s]


In [16]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', 30): 
    display(df_meta.head().T)

,0,1,2,3,4
uid,391,383_1,338,623,559
ct_fname,/mnt/16614710-4b50-44ce-a3...,/mnt/16614710-4b50-44ce-a3...,/mnt/16614710-4b50-44ce-a3...,/mnt/16614710-4b50-44ce-a3...,/mnt/16614710-4b50-44ce-a3...
mask_fname,/mnt/16614710-4b50-44ce-a3...,/mnt/16614710-4b50-44ce-a3...,/mnt/16614710-4b50-44ce-a3...,/mnt/16614710-4b50-44ce-a3...,/mnt/16614710-4b50-44ce-a3...
is_valid,False,False,False,False,False
intent_name,b' ',b' ',b' ',b' ',b' '
sizeof_hdr,348,348,348,348,348
cal_min,0,0,0,0,0
bitpix,16,16,16,16,16
cal_max,0,0,0,0,0
srow_x_0,0,0,0,0,0


In [17]:
# df_meta.to_feather('../metadata/df_meta.fth')

## Lesion Center Point Voxel Locations

In [5]:
df_meta = pd.read_feather('../metadata/df_meta.fth')

As revealed from EDA, we are dealing with a very small amount of lesion data per CT. When training our model, we will need to perform some upsampling on the training data by zooming into these lesions

In [6]:
from modules.dsets import Ct

In [7]:
uid_trn_list = df_meta[df_meta.is_valid==False].uid

In [8]:
lesion_header = ['uid', 'coordI', 'coordR', 'coordC', 'min_index', 'max_index', 'min_row', 'max_row', 'min_column', 'max_column', 'largest_side_px'] 

In [9]:
lesion_locations = []
for uid in tqdm(uid_trn_list):
    ct = Ct(uid)
    lesion_locations = lesion_locations + ct.group_lesions(output_df=False)

  0%|          | 0/199 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] Failed to open local file 'metadata/df_lesions_erosion.fth'. Detail: [errno 2] No such file or directory

In [17]:
len(lesion_locations)

1739

In [18]:
df_lesion_coords = pd.DataFrame(lesion_locations, columns=lesion_header)

In [158]:
df_lesion_coords.head()

,uid,coordI,coordR,coordC,min_index,max_index,min_row,max_row,min_column,max_column,largest_side_px
0,391,15,194,112,15,16,194,196,112,113,2
1,391,27,296,214,25,31,277,316,192,232,40
2,391,36,185,190,36,37,185,187,189,192,3
3,383_1,17,296,358,17,18,293,300,355,362,7
4,383_1,18,302,181,18,19,296,309,176,188,13


In [162]:
df_lesion_coords.drop(df_lesion_coords[df_lesion_coords.coordI < 3].index, inplace=True)
df_lesion_coords.reset_index(drop=True, inplace=True)

In [163]:
df_lesion_coords.to_feather('../metadata/df_lesions.fth')

Come center points contain larger lesions so we can sample around those center points for more data

Let's use the criteria of the largest_side_px to organize the center points for sampling

In [101]:
df_40_to_100 = df_lesion_coords[(df_lesion_coords.largest_side_px > 40) & (df_lesion_coords.largest_side_px < 100)]
df_100_to_200 = df_lesion_coords[(df_lesion_coords.largest_side_px > 100) & (df_lesion_coords.largest_side_px < 200)]
df_large = df_lesion_coords[df_lesion_coords.largest_side_px > 200]

In [102]:
len(df_40_to_100), len(df_100_to_200), len(df_large)

(358, 129, 11)

and create a helper function to sample around the center points using a gaussian distribution

In [84]:
def generate_irc(row, num_points=5):
    
    range_i = row.max_index - row.min_index
    range_r = row.max_row - row.min_row
    range_c = row.max_column - row.min_column
    
    center_i = row.min_index + (range_i//2)
    center_r = row.min_row + (range_r//2)
    center_c = row.min_column + (range_c//2)
    
    new_i_list = np.random.normal(center_i, range_i//3, num_points).clip(row.min_index, row.max_index).astype(np.int)
    new_r_list = np.random.normal(center_r, range_r//3, num_points).clip(row.min_row, row.max_row).astype(np.int)
    new_c_list = np.random.normal(center_c, range_c//3, num_points).clip(row.min_column, row.max_column).astype(np.int)
    
    uid_list = [row.uid] * num_points
    
    return [(uid, i,r,c) for uid, i,r,c in zip(uid_list, new_i_list, new_r_list, new_c_list)]

In [114]:
new_points_list = []

In [115]:
num_points = 3 # 3 new points per center
for _, (idx, row) in tqdm(enumerate(df_40_to_100.iterrows()), total=len(df_40_to_100)):
    new_points_list += generate_irc(row,num_points)

100%|██████████| 358/358 [00:00<00:00, 2144.97it/s]


In [116]:
len(new_points_list)

1074

In [117]:
num_points = 6 # 6 new points per center
for _, (idx, row) in tqdm(enumerate(df_100_to_200.iterrows()), total=len(df_100_to_200)):
    new_points_list += generate_irc(row,num_points)

100%|██████████| 129/129 [00:00<00:00, 1985.30it/s]


In [118]:
len(new_points_list)

1848

In [119]:
num_points = 10 # 10 new points per center
for _, (idx, row) in tqdm(enumerate(df_large.iterrows()), total=len(df_large)):
    new_points_list += generate_irc(row,num_points)

100%|██████████| 11/11 [00:00<00:00, 502.26it/s]


In [120]:
len(new_points_list)

1958

In [122]:
df_lesion_samples = pd.DataFrame(new_points_list, columns =['uid', 'coordI', 'coordR', 'coordC']) 

In [134]:
train_data = pd.concat([df_lesion_samples, df_lesion_coords[['uid', 'coordI', 'coordR', 'coordC']]]).reset_index(drop=True)

Finally, do a bit of clean up and save

In [156]:
train_data.drop(train_data[train_data.coordI < 3].index, inplace=True)
train_data.reset_index(drop=True, inplace=True)

In [157]:
train_data.to_feather('../metadata/df_coords.fth')